# Training Pipeline
Selects features, creates training data, trains and saves model

## Imports

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor, plot_importance
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import hopsworks
from xgboost import XGBRegressor
import joblib
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
import os
from pathlib import Path
import sys
from dotenv import load_dotenv

root_dir = Path().absolute()
# Strip subdirectories if the notebook started in any
if root_dir.parts[-1:] == ('pipeline',):
    root_dir = Path(*root_dir.parts[:-1])
if root_dir.parts[-1:] == ('src',):
    root_dir = Path(*root_dir.parts[:-1])
root_dir = str(root_dir) 

os.chdir(root_dir)
print(f"Root dir: {Path.cwd()}")

from src.data_utils.ingest import *

load_dotenv()
hopsworks_key = os.getenv('HOPSWORKS_API_KEY')

## Connect to Hopsworks Feature Store

In [ ]:
if hopsworks_key is not None:
    os.environ['HOPSWORKS_API_KEY'] = hopsworks_key

# If you are invited to someone else's Hopsworks project, write that project's name below
project_name = None

if project_name:
    project = hopsworks.login(project=f'{project_name}')
else:
    project = hopsworks.login()
    
fs = project.get_feature_store()

# Set up secrets here
secrets = hopsworks.get_secrets_api()
# ...
# ...
# ...

### Retrieve Feature groups

In [ ]:
features_fg = fs.get_feature_group(
    name="delay_features_fg",
    version=1
)

labels_fg = fs.get_feature_group(
    name="delay_labels_fg",
    version=1
)

features_df = features_fg.read(online=True)
labels_df = labels_fg.read(online=True)

In [ ]:
## print info
print("features_df info: ")
print(features_df.info())
print()
print(labels_df.info())

### Merge features and labels into a df

In [ ]:
HORIZON = pd.Timedelta(minutes=20)
TOLERANCE = pd.Timedelta(minutes=20)

In [ ]:
features_df = features_df.reset_index(drop=True)
labels_df = labels_df.reset_index(drop=True)

In [ ]:
features_df = features_df.sort_values(["line", "timestamp"])
labels_df = labels_df.sort_values(["line", "timestamp"])

In [ ]:
features_df["target_time"] = features_df["timestamp"] + HORIZON

In [ ]:
features_df = features_df.sort_values(["line", "target_time"])
labels_df = labels_df.sort_values(["line", "timestamp"])

features_df = features_df.reset_index(drop=True)
labels_df = labels_df.reset_index(drop=True)

In [ ]:
lines = features_df['line'].unique()
merged_list = []
for line in lines:
    features_line = features_df[features_df['line'] == line].sort_values("target_time")
    labels_line = labels_df[labels_df['line'] == line].sort_values("timestamp")
    merged_line = pd.merge_asof(
        features_line,
        labels_line,
        left_on="target_time",
        right_on="timestamp",
        direction="forward",
        tolerance=TOLERANCE,
        suffixes=("_feat", "_label")
    )
    merged_list.append(merged_line)

train_df = pd.concat([merged_list[0], merged_list[1], merged_list[2]], ignore_index=True)
# print(train_df.info())
# print(train_df.head(10))

In [ ]:
train_df = train_df.rename(columns={
    "timestamp_feat": "timestamp",
    "avg_delay": "target", 
    "line_feat": "line",
})

train_df = train_df.drop(columns=[
    "timestamp_str_feat",
    "target_time",
    "timestamp_label",
    "timestamp_str_label"
])

train_df = train_df[["timestamp", "line", "delay_60", "delay_30", "delay_current", "target"]]

In [ ]:
print(train_df.iloc[40])

### Drop rows with nan values

In [ ]:
print(train_df.isna().sum())

In [ ]:
train_df = train_df.dropna(subset=['target'])

In [ ]:
train_df = train_df.dropna(subset=["delay_60"])

In [ ]:
print(train_df.isna().sum())

### add day as feature

In [ ]:
train_df["day"] = train_df["timestamp"].dt.day_name()

In [ ]:
train_df = train_df[["timestamp", "line", "day", "delay_60", "delay_30", "delay_current", "target"]]

In [ ]:
print(train_df.info())

### encode line and day using LabelEncoder

In [ ]:
line_encoder = LabelEncoder()
train_df["line_encoded"] = line_encoder.fit_transform(train_df["line"])

day_encoder = LabelEncoder()
train_df["day_encoded"] = day_encoder.fit_transform(train_df["day"])

if not os.path.exists(Path("model")):
    os.makedirs("model", exist_ok=True)

joblib.dump(line_encoder, "model/line_encoder.pkl")
joblib.dump(day_encoder, "model/day_encoder.pkl")

In [ ]:
train_df = train_df[["timestamp", "line_encoded", "day_encoded", "delay_60", "delay_30", "delay_current", "target"]]
print(train_df.head())

In [ ]:
train_df["timestamp_str"] = train_df["timestamp"].dt.strftime("%Y-%m-%dT%H:%M:%S")

### Split into separate feature and label df (time-compatible)

In [ ]:
features_df = train_df.drop(columns=["target"])
labels_df = train_df[["timestamp", "line_encoded", "target", "timestamp_str"]]

In [ ]:
print(features_df.head(5))
print(labels_df.head(5))

### Create feature view

In [ ]:
features_fg = fs.get_or_create_feature_group(
    name="train_features_fg",
    primary_key=["line_encoded", "timestamp_str"],
    version=1,
    event_time="timestamp",
    online_enabled=True,
    description="Features for delay forecasting"
)

labels_fg = fs.get_or_create_feature_group(
    name="train_labels_fg",
    primary_key=["line_encoded", "timestamp_str"],
    version=1,
    event_time="timestamp",
    online_enabled=True,
    description="Future labels aligned to feature time"
)

# features_fg.insert(features_df, write_options={"wait_for_job": True})
# labels_fg.insert(labels_df, write_options={"wait_for_job": True})

In [ ]:
query = features_fg.select([
    "line_encoded",
    "day_encoded",
    "delay_60",
    "delay_30",
    "delay_current",
    "timestamp"
]).join(
    labels_fg.select(["target"]),
    on=["line_encoded", "timestamp_str"],
)

fv = fs.get_or_create_feature_view(
    name="delay_timeseries_fv",
    version=1,
    description="Feature view for training ML model",
    labels=["target"],
    query=query
)

In [ ]:
# test_start = datetime(2026, 1, 1, 17, 48)
# X_train, X_test, y_train, y_test = fv.train_test_split(test_start=test_start)

### Feature views wont work..., split data and train model manually

In [ ]:
split_time = pd.Timestamp("2026-01-01 23:13:00")
train_data = train_df[train_df['timestamp'] < split_time]
test_data = train_df[train_df['timestamp'] >= split_time]

X_train = train_data.drop(columns=['target', 'timestamp', 'timestamp_str']).reset_index(drop=True)
y_train = train_data['target'].reset_index(drop=True)

X_test = test_data.drop(columns=['target', 'timestamp', 'timestamp_str']).reset_index(drop=True)
y_test = test_data['target'].reset_index(drop=True)

# print(X_train)
# print(y_train)

In [ ]:
xgb_model = XGBRegressor()

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test MSE: {mse:.3f}, R^2: {r2:.3f}")


In [ ]:
mr = project.get_model_registry()
model_name = "xgb_regressor"
joblib.dump(xgb_model, "model/model.joblib")

model = mr.python.create_model(
    name=model_name,
    metrics={"mse": mse, "r2": r2},
    feature_view=fv,
    description="XGBoost regressor for metro delay prediction",
)

model.save("model")

### Plots here! 